# **📹 AUTOMATIC SUBTITLE TEST**


Prueba de subtitulado automático para cumplimiento de las directrices impuestas por la normativa UNE

In [ ]:
!pip install openai-whisper
!pip install requests

In [ ]:
import whisper
import math
import time
import requests
from urllib.parse import urlparse

In [ ]:
# Load model
model = whisper.load_model("medium")

In [ ]:
# Video file urls
media_links = [
  'http://corerepo.acceso.com/api/v2/temp_file/266436c09f1a5be02201bb233116133a/?format=json'
]

### **Helper functions**

In [ ]:

def get_audio_file(media_link):
  response = requests.get(media_link)
  if response.ok:
    audio = response.url

  audio_type = "video" if urlparse(audio).path.endswith("mp4") else "audio"
  return audio, audio_type

In [ ]:
def format_transcription_by_segments(transcription):
  formatted_txt = ""
  for segment in transcription['segments']:
    text = segment['text']
    start_time = float(segment['start'])
    end_time = float(segment['end'])
    diff_secs = end_time-start_time
    cps = math.ceil(len(text.replace(" ", "").strip())/diff_secs)
    formatted_txt += f"{segment['id']}:: {start_time} --> {end_time} ({cps})\n{text}\n\n"
  return formatted_txt

In [ ]:
def format_transcription_by_words(transcription):
  formatted_txt = ""
  for segment in transcription['segments']:
    for word in segment['words']:
      text = word['word']
      start_time = float(word['start'])
      end_time = float(word['end'])
      diff_secs = max(end_time-start_time, 1)
      cps = math.ceil(len(text.replace(" ", "").strip())/diff_secs)
      formatted_txt += f"{start_time} --> {end_time} ({cps})\n{text}\n"
      #formatted_txt = formatted_txt + f"{start_time} --> {end_time} ({cps})\n{text}\n"
  return formatted_txt


### Retrieve audio file to be processed

In [ ]:

audio, audio_type = get_audio_file(media_links[0])

### Test transcription with prompt options

Review:


* [Prompting whisper](https://github.com/openai/openai-cookbook/blob/main/examples/Whisper_prompting_guide.ipynb)
* [Api whisper prompting](https://platform.openai.com/docs/guides/speech-to-text/prompting)
* [Misc prompt engineering whisper](https://medium.com/axinc-ai/prompt-engineering-in-whisper-6bb18003562d)


In [ ]:
# options = dict(beam_size=5, best_of=5)
# transcribe_options = dict(task="transcribe", **options)
# transcription = model.transcribe(audio, **transcribe_options)

PROMPT = """
Genera subtítulos a partir de la trasncripción. Estos subtítulos deben cumplir los siguientes requerimientos impuestos por la norma UNE de subtitulado.
    - No romper sintagmas, deben ir en la misma línea:
        ejemplo: <incorrecto>artículo + sustantivo: el trabajo \n  duro </incorrecto>; <correcto>el trabajo duro </correcto>
        ejemplo: <incorrecto>sustantivo + adjetivo: de \n  manera urgente </incorrecto>; <correcto>de manera urgente<correcto>.
    - No separar verbos compuestos en líneas o subtítulos diferentes:
        ejemplo: </incorrecto>nosotros hemos \n  hecho</incorrecto>; <correcto>nosotros hemos hecho</correcto>.
    -  A final de línea, no pueden quedar determinantes:
        ejemplo: <incorrecto>este viernes en la \n  sede </incorrecto>. <correcto>este viernes en la sede. </correcto>.
    - El máximo de caracteres por línea será 37 caracteres.
    - El máximo de caracteres por segundo (cps) es de 15.
"""
# video = '/content/mda_206247965.mp4'
transcription = model.transcribe(audio)
# transcription = model.transcribe(video, initial_prompt=PROMPT)

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
print(transcription)

In [ ]:
txt = format_transcription_by_segments(transcription)
print(txt)

### Test transcription with word detail options

In [ ]:
transcription2 = model.transcribe(audio, word_timestamps=True)

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
print(transcription2)

{'text': ' también liberado. Y arranca en Barcelona el Mobile, la gran feria de las telecomunicaciones. Se espera que el evento mueva 350 millones de euros, es mucho dinero pero no llega todavía las cifras pre pandemia. Y como es ya habitual, Aragonés y Colau plantaron al rey en la recepción pero luego Gabriela de Torres sí asistieron a la cena. Sí, buenos días. Como bien decís, una vez más se saltaron ese saludo protocolario como gesto de rechazo a la monarquía pero sí que participaron en el cóctel anterior y después en la cena inaugural. En su discurso el monarca quiso ensalzar el valor de esta feria en medio, dijo, de la inestabilidad internacional actual. Ha tenido el rey palabras de aliento para Ucrania en esta inauguración del Mobile y falta hacen en un momento crítico de esta conferencia.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 5.58, 'text': ' también liberado. Y arranca en Barcelona el Mobile, la gran feria de las', 'tokens': [50364, 6407, 6774, 1573, 13, 398, 

In [ ]:
txt = format_transcription_by_words(transcription2)
print(txt)
txt

0.0 --> 0.32 (1)
 o
0.32 --> 0.58 (7)
 también
0.58 --> 1.02 (6)
 podría
1.02 --> 1.44 (5)
 estar
1.44 --> 1.74 (3)
 más
1.74 --> 2.12 (6)
 cerca.
5.66 --> 5.86 (2)
 El
5.86 --> 6.1 (3)
 rey
6.1 --> 6.36 (6)
 Felipe
6.36 --> 6.62 (2)
 VI
6.62 --> 6.82 (2)
 ha
6.82 --> 7.14 (9)
 presidido
7.14 --> 7.32 (2)
 la
7.32 --> 7.46 (4)
 cena
7.46 --> 7.8 (9)
 inaugural
7.8 --> 8.02 (2)
 en
8.02 --> 8.1 (2)
 el
8.1 --> 8.36 (8)
 Congreso
8.36 --> 8.68 (7)
 Mundial
8.68 --> 8.88 (2)
 de
8.88 --> 9.32 (8)
 Móviles,
9.6 --> 9.82 (5)
 donde
9.82 --> 9.98 (2)
 ha
9.98 --> 10.54 (10)
 codificado
10.54 --> 10.74 (3)
 con
10.74 --> 10.94 (2)
 el
10.94 --> 11.42 (10)
 presidente
11.42 --> 11.68 (3)
 del
11.68 --> 12.0 (9)
 gobierno,
12.06 --> 12.28 (5)
 Pedro
12.28 --> 12.66 (8)
 Sánchez,
12.68 --> 12.8 (1)
 y
12.8 --> 12.9 (2)
 el
12.9 --> 13.04 (2)
 de
13.04 --> 13.24 (2)
 la
13.24 --> 13.78 (12)
 Generalitat,
13.96 --> 14.02 (4)
 Pera
14.02 --> 14.38 (9)
 Aragonés.
18.88 --> 19.12 (7)
 También
19.12 -

'0.0 --> 0.32 (1)\n o\n0.32 --> 0.58 (7)\n también\n0.58 --> 1.02 (6)\n podría\n1.02 --> 1.44 (5)\n estar\n1.44 --> 1.74 (3)\n más\n1.74 --> 2.12 (6)\n cerca.\n5.66 --> 5.86 (2)\n El\n5.86 --> 6.1 (3)\n rey\n6.1 --> 6.36 (6)\n Felipe\n6.36 --> 6.62 (2)\n VI\n6.62 --> 6.82 (2)\n ha\n6.82 --> 7.14 (9)\n presidido\n7.14 --> 7.32 (2)\n la\n7.32 --> 7.46 (4)\n cena\n7.46 --> 7.8 (9)\n inaugural\n7.8 --> 8.02 (2)\n en\n8.02 --> 8.1 (2)\n el\n8.1 --> 8.36 (8)\n Congreso\n8.36 --> 8.68 (7)\n Mundial\n8.68 --> 8.88 (2)\n de\n8.88 --> 9.32 (8)\n Móviles,\n9.6 --> 9.82 (5)\n donde\n9.82 --> 9.98 (2)\n ha\n9.98 --> 10.54 (10)\n codificado\n10.54 --> 10.74 (3)\n con\n10.74 --> 10.94 (2)\n el\n10.94 --> 11.42 (10)\n presidente\n11.42 --> 11.68 (3)\n del\n11.68 --> 12.0 (9)\n gobierno,\n12.06 --> 12.28 (5)\n Pedro\n12.28 --> 12.66 (8)\n Sánchez,\n12.68 --> 12.8 (1)\n y\n12.8 --> 12.9 (2)\n el\n12.9 --> 13.04 (2)\n de\n13.04 --> 13.24 (2)\n la\n13.24 --> 13.78 (12)\n Generalitat,\n13.96 --> 14.02 (4)\

Subtitle generation based on word detail.

**Prompt**:

```f
Eres un experto en lengua y subtitulado. El objetivo es crear los subtitulos con su pautado de tiempo a partir de los datos proporcionados. Estos datos son las palabras con su respectivo tiempo de inicio y fin de la mención en un audio. El formato es el siguiente:
---
<tiempo_inicio> --> <tiempo_fin> (<caracteres_por_segundo>)
<palabra>
...
---

Genera subtítulos a partir de los datos que se proporcionen. Estos subtítulos deben cumplir los siguientes requerimientos impuestos por la norma UNE de subtitulado.
    - No romper sintagmas, deben ir en la misma línea:
        ejemplo: <incorrecto>artículo + sustantivo: el trabajo \n  duro </incorrecto>; <correcto>el trabajo duro </correcto>
        ejemplo: <incorrecto>sustantivo + adjetivo: de \n  manera urgente </incorrecto>; <correcto>de manera urgente<correcto>.
    - No separar verbos compuestos en líneas o subtítulos diferentes:
        ejemplo: </incorrecto>nosotros hemos \n  hecho</incorrecto>; <correcto>nosotros hemos hecho</correcto>.
    -  A final de línea, no pueden quedar determinantes:
        ejemplo: <incorrecto>este viernes en la \n  sede </incorrecto>. <correcto>este viernes en la sede. </correcto>.
    - El máximo de caracteres por línea será 37 caracteres.
    - El máximo de caracteres por segundo (cps) es de 15.

DATOS:
---
{FORMATTED_WORD_DETAIL_DATA}
---

```






In [ ]:
def txt_a_diccionario_json(txt):
    subtitulos = txt.split('\n')
    diccionario = []
    for i in range(0, len(subtitulos)-1, 2):
        rango_tiempo, palabra_cps = subtitulos[i], subtitulos[i+1].strip()
        inicio, fin = rango_tiempo.split(" --> ")[0], rango_tiempo.split(" --> ")[1].split()[0]
        cps = rango_tiempo[-2]
        palabra = palabra_cps.split("(")[0].strip()
        diccionario.append({'word': palabra, 'start': inicio, 'end': fin, 'cps': cps})
    return diccionario

In [ ]:
words_dict = txt_a_diccionario_json(txt)

In [ ]:
print(words_dict)